In [9]:
import pandas as pd 
import numpy as np
import random
from citipy import citipy
from datetime import datetime 
import requests
from config_api import weather_apiKey as wkey

In [10]:

# Create list for weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Create basic url for API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + wkey

Beginning Data Retrieval     
-----------------------------


In [11]:
# Create 2000 random latitude and longitude coordinates 
lats = np.random.uniform(-90, 90, size = 2000)
lngs = np.random.uniform(-180, 180, size = 2000)
lats_lngs = zip(lats, lngs)
lats_lngs

# Add coordinates to a list 
latlngs_lst = list(lats_lngs)

In [12]:
# Create list for cities 
cities = []

# Use citipy to find nearest cities using coordinate list
for latlng in latlngs_lst:
    city = citipy.nearest_city(latlng[0],latlng[1])
    city_name = city.city_name
    country_name = city.country_code
    
    # Find unique cities to add to cities list
    if city_name not in cities:
        cities.append(city_name)
    
    # See cities and contries
    # print(f'{city_name}, {country_name}')

# Count amount of unique cities
len(cities)

750

In [13]:

# Loop through city Lists 
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 
    if (i % 50 == 0 and i >= 50):
        set_count += 1 
        record_count = 1 
    
    # Create URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log URL, set, and record numbers for each city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1

# Run an API request for each of the cities
    try:
       
        # Parse the JSON data
        city_weather = requests.get(city_url).json()

        # Compile needed data
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_hum = city_weather['main']['humidity']
        city_cloud = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_desc = city_weather['weather'][0]['description']


        # Fill City Data list 
        city_data.append({"City": city.title(),
                         "Country": city_country,
                         "Date": city_date,
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_hum,
                         "Cloudiness": city_cloud,
                         "Wind Speed": city_wind,
                         "Current Weather": city_desc
                         })
        
    #If an error occurs
    except: 
        print("--- City not found")
        pass

    # Complete Data Retrieval    
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | tiksi
Processing Record 2 of Set 1 | souillac
Processing Record 3 of Set 1 | vostok
Processing Record 4 of Set 1 | luderitz
Processing Record 5 of Set 1 | vaitupu
--- City not found
Processing Record 6 of Set 1 | torbay
Processing Record 7 of Set 1 | mys shmidta
--- City not found
Processing Record 8 of Set 1 | nador
Processing Record 9 of Set 1 | tuktoyaktuk
Processing Record 10 of Set 1 | jamestown
Processing Record 11 of Set 1 | dali
Processing Record 12 of Set 1 | katsuura
Processing Record 13 of Set 1 | victoria
Processing Record 14 of Set 1 | port alfred
Processing Record 15 of Set 1 | pilar
Processing Record 16 of Set 1 | albany
Processing Record 17 of Set 1 | chapais
Processing Record 18 of Set 1 | saint-ambroise
Processing Record 19 of Set 1 | port elizabeth
Processing Record 20 of Set 1 | tabuk
Processing Record 21 of Set 1 | nizhneyansk
--- City not found
Processing Record 22 of Set 1 | kapaa
Processing Record 23 of Set 1 | saskylakh
Processing

In [ ]:
len(city_data)

691

In [ ]:

# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,Evanston,US,2021-07-24 21:36:30,42.0411,-87.6901,91.62,70,40,1.01,thunderstorm
1,Punta Arenas,CL,2021-07-24 21:36:31,-53.1500,-70.9167,41.11,75,75,11.50,broken clouds
2,Chaoyang,CN,2021-07-24 21:36:31,41.5703,120.4586,74.28,87,59,4.61,broken clouds
3,Saint-Philippe,RE,2021-07-24 21:36:31,-21.3585,55.7679,67.59,79,37,4.97,light rain
4,Albany,US,2021-07-24 21:36:31,42.6001,-73.9662,81.25,52,87,3.00,overcast clouds
5,Rikitea,PF,2021-07-24 21:36:32,-23.1203,-134.9692,69.26,54,0,1.43,clear sky
6,Torbay,CA,2021-07-24 21:36:32,47.6666,-52.7314,66.97,79,75,13.80,broken clouds
7,Bluff,NZ,2021-07-24 21:36:32,-46.6000,168.3333,44.80,69,100,10.07,overcast clouds
8,Kapaa,US,2021-07-24 21:36:32,22.0752,-159.3190,82.72,79,75,4.00,light rain
9,Hermanus,ZA,2021-07-24 21:36:32,-34.4187,19.2345,51.91,56,14,5.68,few clouds


In [ ]:
# Create the output file 
output_data_file = '../weather_data/cities.csv'

# Export the City_Data into a CSV
city_data_df.to_csv('WeatherPy_Database.csv', index_label='City_ID')